# Classifying neighborhoods in Athens city center by k-means clustering on the number of parking stations .

The present was created for “Applied Data Science Capstone” course which is part of “Applied Data Science Specialization” e-learning program provided by Coursera in cooperation with IBM.

# Step 1: Before we get the data and start exploring it, we download all the dependencies that we will need.

In [1]:
import numpy as np # Library to handle data in a vectorized manner

import pandas as pd # Library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # Library to handle JSON files

from geopy.geocoders import Nominatim # Convert an address into latitude and longitude values

import requests # Library to handle requests

from pandas.io.json import json_normalize # Tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means for clustering
from sklearn.cluster import KMeans

# Import make_blobs for visulization of municipalities
from sklearn.datasets import make_blobs

import folium # Map rendering library

print('Libraries imported.')

Libraries imported.


# Step 2: We download data for neighborhoods in the city center of Athens from Foursquare and create the map of Athens city center.

Step 2a: We use our Foursquare credentials.

In [1]:
CLIENT_ID = 'CLIENT_ID' # Foursquare ID
CLIENT_SECRET = 'CLIENT_SECRET' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CLIENT_ID
CLIENT_SECRET:CLIENT_SECRET


Step 2b: In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent athens_explorer.

In [3]:
athens_address = 'Athens , Greece'

athens_geolocator = Nominatim(user_agent = "athens_explorer")
athens_location = athens_geolocator.geocode(athens_address)
athens_latitude = athens_location.latitude
athens_longitude = athens_location.longitude

print('The geograpical coordinate of Athens are {}, {}.'.format(athens_latitude , athens_longitude))

The geograpical coordinate of Athens are 37.9839412, 23.7283052.


Step 2c: We define radius and limit of neighborhoods to get.

In [4]:
radius = 300

LIMIT = 30

Step 2d: We create the GET request URL for neighborhoods in Athens city center.

In [5]:
athens_url = 'https://api.foursquare.com/v2/venues/explore?&categoryId=4f2a25ac4b909258e854f55f&client_id={}&client_secret={}&v={}&ll={},{}&radious={}'.format(CLIENT_ID ,
                                                                                                                             CLIENT_SECRET ,
                                                                                                                             VERSION ,
                                                                                                                             athens_latitude ,
                                                                                                                             athens_longitude ,
                                                                                                                             radius ,
                                                                                                                             LIMIT)

athens_url

'https://api.foursquare.com/v2/venues/explore?&categoryId=4f2a25ac4b909258e854f55f&client_id=P0IZSA5JP0VLMJI4TN31HO3Z4QF0DPYXR0MWK5F01DWK3JCO&client_secret=12XWW4MCNJ4IE4GVE4RVFRSGJ0MHU35LIQKBW4QNYFF2QPUK&v=20180605&ll=37.9839412,23.7283052&radious=300'

Step 2e: We send the GET request and examine the results.

In [6]:
athens_neighborhoods_results = requests.get(athens_url).json()

athens_neighborhoods_results

{'meta': {'code': 200, 'requestId': '5ec644fefb34b500286535b7'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 600,
  'headerLocation': 'Athina 1',
  'headerFullLocation': 'Athina 1, Athens',
  'headerLocationGranularity': 'neighborhood',
  'query': 'neighborhood',
  'totalResults': 22,
  'suggestedBounds': {'ne': {'lat': 37.99896498189817,
    'lng': 23.744198016908815},
   'sw': {'lat': 37.971212554193684, 'lng': 23.71123335328893}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c6c058cd08695217cc602ea',
       'name': 'Lofos Strefi (Λόφος Στρέφη)',
       'location': {'lat': 37.98797480510275,
        'lng': 23.738003521868677,
        'labeledLatLngs': [{'label': 'display'

Step 2f: We clean the json and structure it into a pandas dataframe.

In [7]:
athens_neighborhoods = athens_neighborhoods_results['response']['groups'][0]['items']
    
athens_center = json_normalize(athens_neighborhoods) # Flatten JSON

# Filter column
athens_neighborhoods_columns = ['venue.name' , 'venue.location.lat' , 'venue.location.lng']
athens_center = athens_center.loc[: , athens_neighborhoods_columns]

# Clean columns
athens_center.columns = [col.split(".")[-1] for col in athens_center.columns]
                         
athens_center

,name,lat,lng
0,Lofos Strefi (Λόφος Στρέφη),37.987975,23.738004
1,Omonoia (Ομόνοια),37.984194,23.728503
2,Kapnikareas Square (Πλατεία Καπνικαρέας),37.976312,23.728692
3,Akadimia (Ακαδημία),37.981125,23.733117
4,Metaxourgeio (Μεταξουργείο),37.985889,23.722645
5,Psyrri (Ψυρρή),37.978439,23.724986
6,Exarcheia (Εξάρχεια),37.986144,23.735487
7,Monastiraki Square (Πλατεία Μοναστηρακίου),37.976108,23.725814
8,Monastiraki (Μοναστηράκι),37.976715,23.726306
9,Pefkakia Lykavittou (Πευκάκια Λυκαβηττού),37.984003,23.739073


Step 2g: We rename the columns in the dataframe which contains neighborhoods in Athens city center.

In [8]:
athens_center.rename(columns = {'name' : 'Neighborhood' , 'lat' : 'NeighborhoodLat' , 'lng' : 'NeighborhoodLong'} , inplace = True)

athens_center

,Neighborhood,NeighborhoodLat,NeighborhoodLong
0,Lofos Strefi (Λόφος Στρέφη),37.987975,23.738004
1,Omonoia (Ομόνοια),37.984194,23.728503
2,Kapnikareas Square (Πλατεία Καπνικαρέας),37.976312,23.728692
3,Akadimia (Ακαδημία),37.981125,23.733117
4,Metaxourgeio (Μεταξουργείο),37.985889,23.722645
5,Psyrri (Ψυρρή),37.978439,23.724986
6,Exarcheia (Εξάρχεια),37.986144,23.735487
7,Monastiraki Square (Πλατεία Μοναστηρακίου),37.976108,23.725814
8,Monastiraki (Μοναστηράκι),37.976715,23.726306
9,Pefkakia Lykavittou (Πευκάκια Λυκαβηττού),37.984003,23.739073


Step 2h: Create a map of the neighborhoods in Athens city center.

In [9]:
athens_map = folium.Map(location=[athens_latitude, athens_longitude], zoom_start = 14)

# Add markers to map
for neighborhood , lat , long in zip(athens_center['Neighborhood'] , athens_center['NeighborhoodLat'] , athens_center['NeighborhoodLong']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, long],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(athens_map)  
    
athens_map

In [10]:
print('{} neighborhoods in Athens city center were returned by Foursquare.'.format(athens_center.shape[0]))

22 neighborhoods in Athens city center were returned by Foursquare.


# Step 3: We find parkings in Athens city centre from Foursquare.

Step 3a: We define radius and limit of parkings to get.

In [11]:
rad = 300

LIM = 30

Step 3b: We create the GET request URL for parkings in Athens city center.

In [12]:
athens_parkings_url = 'https://api.foursquare.com/v2/venues/explore?&categoryId=4c38df4de52ce0d596b336e1&client_id={}&client_secret={}&v={}&ll={},{}&radious={}'.format(CLIENT_ID ,
                                                                                                                             CLIENT_SECRET ,
                                                                                                                             VERSION ,
                                                                                                                             athens_latitude ,
                                                                                                                             athens_longitude ,
                                                                                                                             rad ,
                                                                                                                             LIM)

athens_parkings_url

'https://api.foursquare.com/v2/venues/explore?&categoryId=4c38df4de52ce0d596b336e1&client_id=P0IZSA5JP0VLMJI4TN31HO3Z4QF0DPYXR0MWK5F01DWK3JCO&client_secret=12XWW4MCNJ4IE4GVE4RVFRSGJ0MHU35LIQKBW4QNYFF2QPUK&v=20180605&ll=37.9839412,23.7283052&radious=300'

Step 3c: We send the GET request and examine the results of parkings.

In [13]:
athens_parkings_results = requests.get(athens_parkings_url).json()

athens_parkings_results

{'meta': {'code': 200, 'requestId': '5ec6457ec94979001bf3868d'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 600,
  'headerLocation': 'Athina 1',
  'headerFullLocation': 'Athina 1, Athens',
  'headerLocationGranularity': 'neighborhood',
  'query': 'parking',
  'totalResults': 47,
  'suggestedBounds': {'ne': {'lat': 37.99532155383226,
    'lng': 23.744547691164396},
   'sw': {'lat': 37.97222898162284, 'lng': 23.7210218393007}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ec0db979a52cce4f3ff23ba',
       'name': 'Komerax Parking',
       'location': {'address': 'Χαλκοκονδύλη 40-42',
        'lat': 37.986066857629005,
        'lng': 23.728162050247192,
        'labeledLatLngs

Step 3d: We clean the json and structure it into a pandas dataframe.

In [14]:
parkings = athens_parkings_results['response']['groups'][0]['items']
    
nearby_ath_parkings = json_normalize(parkings) # Flatten JSON

# Filter columns
filtered_columns = ['venue.name' ,  'venue.location.lat' , 'venue.location.lng']#'venue.categories' ,
nearby_ath_parkings = nearby_ath_parkings.loc[: , filtered_columns]

# Clean columns
nearby_ath_parkings.columns = [col.split(".")[-1] for col in nearby_ath_parkings.columns]

nearby_ath_parkings

,name,lat,lng
0,Komerax Parking,37.986067,23.728162
1,Parking Corfu,37.984959,23.725812
2,Polis Park,37.985379,23.731132
3,Titania Parking,37.983149,23.731600
4,Parking Athens Komerax,37.986791,23.728004
5,Kotzia Square Parking,37.980943,23.728104
6,Ελληνικοι Σταθμοι Αυτ/των,37.984400,23.723157
7,Υπόγειο πάρκινγκ πλατείας κλαυθμώνος,37.979588,23.730927
8,Παρκινγκ Ιπποκράτους 33 & Σόλωνος,37.982609,23.735197
9,Parking Paradise,37.981926,23.735268


Step 3e: We rename columns in nearby_ath_parkings dataframe.

In [15]:
nearby_ath_parkings.rename(columns = {'name' : 'Parking' , 'lat' : 'ParkingLat' , 'lng' : 'ParkingLong'} , inplace = True)

nearby_ath_parkings

,Parking,ParkingLat,ParkingLong
0,Komerax Parking,37.986067,23.728162
1,Parking Corfu,37.984959,23.725812
2,Polis Park,37.985379,23.731132
3,Titania Parking,37.983149,23.731600
4,Parking Athens Komerax,37.986791,23.728004
5,Kotzia Square Parking,37.980943,23.728104
6,Ελληνικοι Σταθμοι Αυτ/των,37.984400,23.723157
7,Υπόγειο πάρκινγκ πλατείας κλαυθμώνος,37.979588,23.730927
8,Παρκινγκ Ιπποκράτους 33 & Σόλωνος,37.982609,23.735197
9,Parking Paradise,37.981926,23.735268


Step 3f: We create a new dataframe called athens_parkings.

In [20]:
def getNearbyParkings(names , latitudes , longitudes , radius = 300):
    
    ath_parkings_list=[]
    
    for name , lat , lng in zip(names , latitudes , longitudes):
        print(name)
            
        # Create the API request URL
        ath_park_url = 'https://api.foursquare.com/v2/venues/explore?&categoryId=4c38df4de52ce0d596b336e1&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID , 
            CLIENT_SECRET , 
            VERSION , 
            lat , 
            long , 
            rad , 
            LIM)
            
        # Make the GET request
        ath_park_results = requests.get(ath_park_url).json()['response']['groups'][0]['items']
        
        # Return only relevant information for each nearby venue
        ath_parkings_list.append([(
            name , 
            lat , 
            long , 
            v['venue']['name'] , 
            v['venue']['location']['lat'] , 
            v['venue']['location']['lng'] , #for v in ath_park_results])   
            v['venue']['categories'][0]['name']) for v in ath_park_results])

    nearby_parkings = pd.DataFrame([item for ath_parkings_list in ath_parkings_list for item in ath_parkings_list])
    nearby_parkings.columns = ['Neighborhood' , 
                  'NeighborhoodLat' , 
                  'NeighborhoodLong' , 
                  'Parking' , 
                  'Parking Lat' , 
                  'Parking Long' ,
                  'Parking Category']
    
    return(nearby_parkings)

In [21]:
athens_parkings = getNearbyParkings(names = athens_center['Neighborhood'] , 
                                    latitudes = athens_center['NeighborhoodLat'] , 
                                    longitudes = athens_center['NeighborhoodLong']
                                   )

Lofos Strefi (Λόφος Στρέφη)
Omonoia (Ομόνοια)
Kapnikareas Square (Πλατεία Καπνικαρέας)
Akadimia (Ακαδημία)
Metaxourgeio (Μεταξουργείο)
Psyrri (Ψυρρή)
Exarcheia (Εξάρχεια)
Monastiraki Square (Πλατεία Μοναστηρακίου)
Monastiraki (Μοναστηράκι)
Pefkakia Lykavittou (Πευκάκια Λυκαβηττού)
Syntagma Square (Πλατεία Συντάγματος)
Pedion Areos (Πεδίον Άρεως)
Anafiotika (Αναφιώτικα)
Syntagma (Σύνταγμα)
Stathmos Larisis (Σταθμός Λαρίσης)
Plaka (Πλάκα)
Thissio (Θησείο)
Neapoli (Νεάπολη)
Kolonaki (Κολωνάκι)
Victoria (Βικτώρια)
Kerameikos (Κεραμεικός)
Agios Panteleimonas (Άγιος Παντελεήμονας)


In [22]:
athens_parkings

,Neighborhood,NeighborhoodLat,NeighborhoodLong,Parking,Parking Lat,Parking Long,Parking Category
0,Lofos Strefi (Λόφος Στρέφη),37.987975,23.729671,Parking Athens Komerax,37.986791,23.728004,Parking
1,Lofos Strefi (Λόφος Στρέφη),37.987975,23.729671,Komerax Parking,37.986067,23.728162,Parking
2,Omonoia (Ομόνοια),37.984194,23.729671,Polis Park,37.985379,23.731132,Parking
3,Omonoia (Ομόνοια),37.984194,23.729671,Titania Parking,37.983149,23.731600,Parking
4,Omonoia (Ομόνοια),37.984194,23.729671,Komerax Parking,37.986067,23.728162,Parking
5,Kapnikareas Square (Πλατεία Καπνικαρέας),37.976312,23.729671,metropark,37.974941,23.731355,Parking
6,Kapnikareas Square (Πλατεία Καπνικαρέας),37.976312,23.729671,Automedon Park,37.976679,23.730136,Parking
7,Kapnikareas Square (Πλατεία Καπνικαρέας),37.976312,23.729671,City Unity College,37.977362,23.730796,College Academic Building
8,Kapnikareas Square (Πλατεία Καπνικαρέας),37.976312,23.729671,ParkAround - Εκπτωση στο Parking,37.976837,23.731398,Tech Startup
9,Akadimia (Ακαδημία),37.981125,23.729671,Kotzia Square Parking,37.980943,23.728104,Parking


Step 3g: We check how many parkings were returned for each neighborhood.

In [23]:
athens_parkings.groupby('Neighborhood').count().reset_index()

,Neighborhood,NeighborhoodLat,NeighborhoodLong,Parking,Parking Lat,Parking Long,Parking Category
0,Akadimia (Ακαδημία),4,4,4,4,4,4
1,Anafiotika (Αναφιώτικα),4,4,4,4,4,4
2,Exarcheia (Εξάρχεια),3,3,3,3,3,3
3,Kapnikareas Square (Πλατεία Καπνικαρέας),4,4,4,4,4,4
4,Kerameikos (Κεραμεικός),3,3,3,3,3,3
5,Kolonaki (Κολωνάκι),4,4,4,4,4,4
6,Lofos Strefi (Λόφος Στρέφη),2,2,2,2,2,2
7,Metaxourgeio (Μεταξουργείο),3,3,3,3,3,3
8,Monastiraki (Μοναστηράκι),4,4,4,4,4,4
9,Monastiraki Square (Πλατεία Μοναστηρακίου),4,4,4,4,4,4


# Step 4: We analyze each neighborhood.

Step 4a: We analyze each neighborhood in Athens city center.

In [24]:
# One hot encoding
ath_onehot = pd.get_dummies(athens_parkings[['Parking Category']] , prefix="" , prefix_sep="")

# Add neighborhood column back to dataframe
ath_onehot['Neighborhood'] = athens_parkings['Neighborhood'] 

# Move neighborhood column to the first column
fixed_columns = [ath_onehot.columns[-1]] + list(ath_onehot.columns[:-1])
ath_onehot = ath_onehot[fixed_columns]

ath_onehot

,Neighborhood,College Academic Building,Hotel,Parking,Tech Startup
0,Lofos Strefi (Λόφος Στρέφη),0,0,1,0
1,Lofos Strefi (Λόφος Στρέφη),0,0,1,0
2,Omonoia (Ομόνοια),0,0,1,0
3,Omonoia (Ομόνοια),0,0,1,0
4,Omonoia (Ομόνοια),0,0,1,0
5,Kapnikareas Square (Πλατεία Καπνικαρέας),0,0,1,0
6,Kapnikareas Square (Πλατεία Καπνικαρέας),0,0,1,0
7,Kapnikareas Square (Πλατεία Καπνικαρέας),1,0,0,0
8,Kapnikareas Square (Πλατεία Καπνικαρέας),0,0,0,1
9,Akadimia (Ακαδημία),0,0,1,0


Step 4b: We group rows by neighborhood and by taking the mean of the frequency of occurrence of each parking.

In [25]:
ath_grouped = ath_onehot.groupby('Neighborhood').mean().reset_index()

ath_grouped

,Neighborhood,College Academic Building,Hotel,Parking,Tech Startup
0,Akadimia (Ακαδημία),0.000000,0.25,0.750000,0.00
1,Anafiotika (Αναφιώτικα),0.000000,0.00,1.000000,0.00
2,Exarcheia (Εξάρχεια),0.000000,0.00,1.000000,0.00
3,Kapnikareas Square (Πλατεία Καπνικαρέας),0.250000,0.00,0.500000,0.25
4,Kerameikos (Κεραμεικός),0.333333,0.00,0.666667,0.00
5,Kolonaki (Κολωνάκι),0.250000,0.00,0.500000,0.25
6,Lofos Strefi (Λόφος Στρέφη),0.000000,0.00,1.000000,0.00
7,Metaxourgeio (Μεταξουργείο),0.000000,0.00,1.000000,0.00
8,Monastiraki (Μοναστηράκι),0.250000,0.00,0.500000,0.25
9,Monastiraki Square (Πλατεία Μοναστηρακίου),0.250000,0.00,0.500000,0.25


# Step 5: We classify neighborhoods in Athens city center by k-means clustering.

Step 5a: We run k-means to cluster the neighborhood into 5 clusters.

In [26]:
# Set number of clusters

k = 5

ath_grouped_clustering = ath_grouped.drop('Neighborhood' , 1)

# Run k-means clustering
kmeans = KMeans(n_clusters = k, random_state = 0).fit(ath_grouped_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_

# Add clustering labels
ath_grouped.insert(0, 'Cluster', kmeans.labels_)

athens_merged = athens_center

# Merge ath_grouped with athens_center to add latitude/longitude for each neighborhood
athens_merged = athens_merged.join(ath_grouped.set_index('Neighborhood'), on = 'Neighborhood')

athens_merged

,Neighborhood,NeighborhoodLat,NeighborhoodLong,Cluster,College Academic Building,Hotel,Parking,Tech Startup
0,Lofos Strefi (Λόφος Στρέφη),37.987975,23.738004,0.0,0.000000,0.00,1.000000,0.00
1,Omonoia (Ομόνοια),37.984194,23.728503,0.0,0.000000,0.00,1.000000,0.00
2,Kapnikareas Square (Πλατεία Καπνικαρέας),37.976312,23.728692,1.0,0.250000,0.00,0.500000,0.25
3,Akadimia (Ακαδημία),37.981125,23.733117,3.0,0.000000,0.25,0.750000,0.00
4,Metaxourgeio (Μεταξουργείο),37.985889,23.722645,0.0,0.000000,0.00,1.000000,0.00
5,Psyrri (Ψυρρή),37.978439,23.724986,1.0,0.250000,0.00,0.500000,0.25
6,Exarcheia (Εξάρχεια),37.986144,23.735487,0.0,0.000000,0.00,1.000000,0.00
7,Monastiraki Square (Πλατεία Μοναστηρακίου),37.976108,23.725814,1.0,0.250000,0.00,0.500000,0.25
8,Monastiraki (Μοναστηράκι),37.976715,23.726306,1.0,0.250000,0.00,0.500000,0.25
9,Pefkakia Lykavittou (Πευκάκια Λυκαβηττού),37.984003,23.739073,0.0,0.000000,0.00,1.000000,0.00


Step 5b: We drop NaN values in athens_merged dataframe.

In [27]:
athens_merged = athens_merged.dropna()

athens_merged

,Neighborhood,NeighborhoodLat,NeighborhoodLong,Cluster,College Academic Building,Hotel,Parking,Tech Startup
0,Lofos Strefi (Λόφος Στρέφη),37.987975,23.738004,0.0,0.000000,0.00,1.000000,0.00
1,Omonoia (Ομόνοια),37.984194,23.728503,0.0,0.000000,0.00,1.000000,0.00
2,Kapnikareas Square (Πλατεία Καπνικαρέας),37.976312,23.728692,1.0,0.250000,0.00,0.500000,0.25
3,Akadimia (Ακαδημία),37.981125,23.733117,3.0,0.000000,0.25,0.750000,0.00
4,Metaxourgeio (Μεταξουργείο),37.985889,23.722645,0.0,0.000000,0.00,1.000000,0.00
5,Psyrri (Ψυρρή),37.978439,23.724986,1.0,0.250000,0.00,0.500000,0.25
6,Exarcheia (Εξάρχεια),37.986144,23.735487,0.0,0.000000,0.00,1.000000,0.00
7,Monastiraki Square (Πλατεία Μοναστηρακίου),37.976108,23.725814,1.0,0.250000,0.00,0.500000,0.25
8,Monastiraki (Μοναστηράκι),37.976715,23.726306,1.0,0.250000,0.00,0.500000,0.25
9,Pefkakia Lykavittou (Πευκάκια Λυκαβηττού),37.984003,23.739073,0.0,0.000000,0.00,1.000000,0.00


Step 5c: We convert data in 'Cluster' column as integer. 

In [28]:
athens_merged['Cluster'] = athens_merged.Cluster.astype(int)

athens_merged

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Neighborhood,NeighborhoodLat,NeighborhoodLong,Cluster,College Academic Building,Hotel,Parking,Tech Startup
0,Lofos Strefi (Λόφος Στρέφη),37.987975,23.738004,0,0.000000,0.00,1.000000,0.00
1,Omonoia (Ομόνοια),37.984194,23.728503,0,0.000000,0.00,1.000000,0.00
2,Kapnikareas Square (Πλατεία Καπνικαρέας),37.976312,23.728692,1,0.250000,0.00,0.500000,0.25
3,Akadimia (Ακαδημία),37.981125,23.733117,3,0.000000,0.25,0.750000,0.00
4,Metaxourgeio (Μεταξουργείο),37.985889,23.722645,0,0.000000,0.00,1.000000,0.00
5,Psyrri (Ψυρρή),37.978439,23.724986,1,0.250000,0.00,0.500000,0.25
6,Exarcheia (Εξάρχεια),37.986144,23.735487,0,0.000000,0.00,1.000000,0.00
7,Monastiraki Square (Πλατεία Μοναστηρακίου),37.976108,23.725814,1,0.250000,0.00,0.500000,0.25
8,Monastiraki (Μοναστηράκι),37.976715,23.726306,1,0.250000,0.00,0.500000,0.25
9,Pefkakia Lykavittou (Πευκάκια Λυκαβηττού),37.984003,23.739073,0,0.000000,0.00,1.000000,0.00


# Step 6: We visualize the resulting clusters by creating a map.

In [29]:
# Create map
map_clusters = folium.Map(location = [athens_latitude , athens_longitude] , zoom_start = 14)

# Set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0 , 1 , len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(athens_merged['NeighborhoodLat'], athens_merged['NeighborhoodLong'], athens_merged['Neighborhood'], athens_merged['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster) , parse_html = True)
    folium.CircleMarker(
        [lat , lon] ,
        radius = 5 ,
        popup = label ,
        color = rainbow[cluster-1] ,
        fill = True ,
        fill_color = rainbow[cluster-1] ,
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters